In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras import layers

## Load the data

In [ ]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train = train_df.drop('label', axis=1)
target = train_df.label

In [ ]:
train.shape

In [ ]:
train.isnull().sum().sum()

## Visualize the data

In [ ]:
import seaborn as sns

sns.countplot(target)

# Pre-processing for CNN

In [ ]:
train_reshaped = train.values.reshape(-1, 28, 28, 1)
train_reshaped.shape

In [ ]:
target.shape

In [ ]:
# Target encoding
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical

target_reshaped = to_categorical(target, 10)
target_reshaped.shape

## Train and test split

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_reshaped, target_reshaped, test_size=0.2, random_state=42, stratify=target_reshaped)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
test_df.shape

In [ ]:
X_test_reshaped = test_df.values.reshape(-1, 28, 28, 1)
X_test_reshaped.shape

# Modelling CNN

## First model CNN- model1

- with MaxPooling, Drropout, Flatten, Early stopping, callback
- score in leaderboard Score: 0.99103

In [ ]:
model1 = tf.keras.Sequential()

# First layer
model1.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(28,28,1), padding='same' ))
model1.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same' ))
model1.add(tf.keras.layers.Dropout(0.25))
model1.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

# Second
model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', padding='same' ) )
model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', padding='same' ) )
model1.add(tf.keras.layers.Dropout(0.25))
model1.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

# Flatten
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(256, activation='relu' )) # 10 iba pri vystupnej vrstve, tu vacsie cislo (256)
model1.add(tf.keras.layers.Dropout(0.25))
model1.add(tf.keras.layers.Dense(10, activation='softmax' ))

In [ ]:
anhealer = tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, min_lr=0.0001) # Reduce learning rate when a metric has stopped improving.
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)
callback=[anhealer, early_stopping]

In [ ]:
model1.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['acc'])   
model1.summary()

In [ ]:
model1.fit(X_train, y_train,
          validation_data = (X_val, y_val),
          batch_size=124,
          epochs=22,
          callbacks=callback
          ) # batch 64-128

# Prediction

In [ ]:
pred1 = model1.predict(X_test_reshaped)

In [ ]:
pred1.shape

In [ ]:
pred1[0]

In [ ]:
pred1[0].argmax()

## Submission

In [ ]:
#submit your work

classes = pred1.argmax(axis=-1)

sub = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sub.Label = classes
sub.head()

In [ ]:
sub.to_csv('submission.csv', index = False)

In [ ]:
sub_df = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
sub_df